In [1]:
%reload_ext autoreload
%autoreload 2

import asyncio
import json
import nest_asyncio
import os
import sys
from dotenv import load_dotenv
import numpy as np
import pandas as pd

sys.path.append('../')
from lattereview.providers.openai_provider import OpenAIProvider
from lattereview.providers.ollama_provider import OllamaProvider
from lattereview.providers.litellm_provider import LiteLLMProvider
from lattereview.agents.scoring_reviewer import ScoringReviewer
from lattereview.review_workflow import ReviewWorkflow

Failed to update token costs. Using static costs.
/Users/pouria/Documents/Coding/LatteReview/venv/lib/python3.9/site-packages/tokencost/constants.py:69: RuntimeWarning: coroutine 'update_token_costs' was never awaited
  logger.error("Failed to update token costs. Using static costs.")
/Users/pouria/Documents/Coding/LatteReview/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Setting up the notebook

Loading environment variables:

In [2]:
# Load environment variables from .env file
load_dotenv('../.env')
print(os.getenv('OPENAI_API_KEY'))

# Enable asyncio in Jupyter
nest_asyncio.apply()

sk-cq_M0pNgHhCFnlDOCMnagYA1l2X7Yea5CL0ci5pZMNT3BlbkFJ0m0x9wm5M_EstX5SjLu_kdwGMDYjkUdviNPs4pe9cA


Loading a dummy dataset:

In [3]:
data = pd.read_excel('data.xlsx')
data.head()

,ID,Title,1st author,repo,year,abstract
0,1,Segmentized quarantine policy for managing a t...,"Kim, J.",arXiv,2024,"By the end of 2021, COVID-19 had spread to ove..."
1,2,AutoProteinEngine: A Large Language Model Driv...,"Liu, Y.",arXiv,2024,Protein engineering is important for biomedica...
2,3,Integration of Large Vision Language Models fo...,"Chen, Z.",arXiv,2024,Traditional natural disaster response involves...
3,4,Choice between Partial Trajectories,"Marklund, H.",arXiv,2024,As AI agents generate increasingly sophisticat...
4,5,Building Altruistic and Moral AI Agent with Br...,"Zhao, F.",arXiv,2024,"As AI closely interacts with human society, it..."


## Testing the base functionalities

Testing the OpenAI provider (with OpenAI and Gemini models):

In [4]:
# openanai_provider = OpenAIProvider(model="gpt-4o-mini")
openanai_provider = OpenAIProvider(model="gemini-1.5-flash")
question = "What is the capital of France?"
asyncio.run(openanai_provider.get_response(question))

('The capital of France is Paris.\n',
 {'input_cost': 5.25e-07, 'output_cost': 2.1e-06, 'total_cost': 2.625e-06})

Testing the Ollama provider:

In [5]:
ollama_provider = OllamaProvider(model="llama3.2-vision:latest", host="http://localhost:11434")
question = "What is the capital of France?"
asyncio.run(ollama_provider.get_response(question))

('The capital of France is Paris.',
 {'input_cost': 0, 'output_cost': 0, 'total_cost': 0})

Testing the LiteLLM provider:

In [6]:
# litellm_provider = LiteLLMProvider(model="gpt-4o-mini")
# litellm_provider = LiteLLMProvider(model="claude-3-5-sonnet-20240620")
# litellm_provider = LiteLLMProvider(model="groq/llama-3.3-70b-versatile")
# litellm_provider = LiteLLMProvider(model="ollama/llama3.2-vision:latest")
litellm_provider = LiteLLMProvider(model="groq/llama-3.3-70b-versatile")

question = "What is the capital of France?"
asyncio.run(litellm_provider.get_response(question))

response: <Response [200 OK]>


('The capital of France is Paris.', 3.464e-05)

Testing the ScoringReviewer agent:

In [7]:
agent = ScoringReviewer(
    # provider=OpenAIProvider(model="gpt-4o-mini"),
    # provider=OpenAIProvider(model="gemini-1.5-flash"),
    # provider=OllamaProvider(model="llama3.2-vision:latest", host="http://localhost:11434"),
    # provider=LiteLLMProvider(model="gpt-4o-mini"),
    provider=LiteLLMProvider(model="claude-3-5-sonnet-20241022"),
    # provider=LiteLLMProvider(model="groq/llama-3.3-70b-versatile"),
    # provider=LiteLLMProvider(model="gemini/gemini-1.5-flash"),
    name="Pouria",
    max_concurrent_requests=1, 
    backstory="an expert reviewer and researcher!",
    input_description = "article title",
    model_args={"max_tokens": 100, "temperature": 0.1},
    reasoning = "brief",
    review_criteria="Look for articles that certainly do not employ any AI or machine learning agents",
    score_set=[1, 2],
    scoring_rules='Score 1 if the paper does not meet the criteria, and 2 if the paper meets the criteria.',
)


# Dummy input
text_list = data.Title.str.lower().tolist()
print("Inputs:\n\n", '\n'.join(text_list[:3]), "\n\n")

# Dummy review
results, total_cost = asyncio.run(agent.review_items(text_list[:3]))
print("Outputs:")
for result in results:
    print(result)

# Dummy costs
print("\nCosts:\n")
for item in agent.memory:
    print(item['cost'])

print("\nTotal cost:\n")
print(total_cost)

Inputs:

 segmentized quarantine policy for managing a tradeoff between containment of infectious disease and social cost of quarantine
autoproteinengine: a large language model driven agent framework for multimodal automl in protein engineering
integration of large vision language models for efficient post-disaster damage assessment and reporting 




Reviewing 3 items - 2024-12-14 20:37:42: 100%|██████████| 3/3 [00:06<00:00,  2.04s/it]

Outputs:
{'reasoning': 'The title clearly indicates a policy-focused study that addresses the balance between disease containment and social impacts through quarantine segmentation, which is a relevant and well-defined research topic.', 'score': 2}
{'reasoning': 'The title clearly indicates a framework for protein engineering that leverages large language models and multimodal AutoML, suggesting a novel and relevant contribution to the field.', 'score': 2}
{'reasoning': 'The title effectively indicates the use of vision-language models for post-disaster assessment, which is a clear and relevant application of AI technology for disaster management.', 'score': 2}

Costs:

0.003057
0.0029760000000000003
0.00294

Total cost:

0.00294


## Testing the main Functionalities

#### A multiagent review workflow for doing title/abstract analysis

Setting up the agents:

In [8]:
pouria = ScoringReviewer(
    # provider=OpenAIProvider(model="gemini-1.5-flash"),
    # provider=OllamaProvider(model="llama3.2-vision:latest", host="http://localhost:11434"),
    # provider=LiteLLMProvider(model="groq/llama-3.3-70b-versatile"),
    # provider=LiteLLMProvider(model="groq/llama-3.3-70b-versatile"),
    provider=LiteLLMProvider(model="gemini/gemini-1.5-flash"),
    name="Pouria",
    max_concurrent_requests=20, 
    backstory="a radiologist with many years of background in statistcis and data science, who are famous among your colleagues for your systematic thinking, organizaton of thoughts, and being conservative",
    model_args={"max_tokens": 100, "temperature": 0.1},
    input_description = "tilte and abstract of scientific articles",
    reasoning = "cot",
    scoring_task="Look for articles that disucss large languange models-based AI agents applied to medical imaging data",
    score_set=[1, 2],
    scoring_rules='Score 1 if the paper meets the criteria, and 2 if the paper does not meet the criteria.',
)

bardia = ScoringReviewer(
    provider=OpenAIProvider(model="gpt-4o-mini"),
    name="Bardia",
    max_concurrent_requests=20, 
    backstory="an expert in data science with a background in developing ML models for healthcare, who are famous among your colleagues for your creativity and out of the box thinking",
    model_args={"max_tokens": 100, "temperature": 0.8},
    input_description = "tilte and abstract of scientific articles",
    reasoning = "brief",
    scoring_task="Look for articles that disucss large languange models-based AI agents applied to medical imaging data",
    score_set=[1, 2],
    scoring_rules='Score 1 if the paper meets the criteria, and 2 if the paper does not meet the criteria.',
)

brad = ScoringReviewer(
    provider=OpenAIProvider(model="gpt-4o"),
    name="Brad",
    max_concurrent_requests=20, 
    backstory="a senior radiologist with a PhD in computer science and years of experience as the director of a DL lab focused on developing ML models for radiology and healthcare",
    input_description = "tilte and abstract of scientific articles",
    temperature=0.4,
    reasoning = "cot",
    max_tokens=100,
    scoring_task="""Pouria and Bardia have Looked for articles that disucss large languange models-based AI agents applied to medical imaging data. 
                       They scored an article 1 if they thought it does not meet this criteria, 2 if they thought it meets the criteria, 0 if they were uncertain of scoring.
                       You will receive an article they have had different opinions about, as well as each of their scores and their reasoning for that score. Read their reviews and determine who you agree with. 
                    """,
    score_set=[1, 2],
    scoring_rules="""Score 1 if you agree with Pouria, and score 2 if you agree with Bardia.""",
)


Setting up the review workflow:

In [9]:
title_abs_review = ReviewWorkflow(
    workflow_schema=[
        {
            "round": 'A',
            "reviewers": [pouria, bardia],
            "inputs": ["Title", "abstract"]
        },
        {
            "round": 'B',
            "reviewers": [brad],
            "inputs": ["Title", "abstract", "round-A_Pouria_output", "round-A_Bardia_output"],
            "filter": lambda row: row["round-A_Pouria_output"]["score"] != row["round-A_Bardia_output"]["score"]
        }
    ]
)

Applying the review workflow to a number of sample articles:

In [10]:
# Reload the data if needed.
sample_data = pd.read_excel('data.xlsx').sample(10).reset_index(drop=True)
updated_data = asyncio.run(title_abs_review(sample_data))

print("Total cost: ")
print(title_abs_review.get_total_cost())

print("\nDetailed cost:")
print(title_abs_review.reviewer_costs)

updated_data


Starting review round A (1/2)...
Processing 10 eligible rows


['round: A', 'reviewer_name: Pouria'] - 2024-12-14 20:09:54: 100%|██████████| 10/10 [00:01<00:00,  6.03it/s]
['round: A', 'reviewer_name: Bardia'] - 2024-12-14 20:09:55: 100%|██████████| 10/10 [00:01<00:00,  6.74it/s]



Starting review round B (2/2)...
Processing 2 eligible rows


['round: B', 'reviewer_name: Brad'] - 2024-12-14 20:09:57: 100%|██████████| 2/2 [00:02<00:00,  1.41s/it]

Total cost: 
0.00421395

Detailed cost:
{('A', 'Pouria'): 6.555e-05, ('A', 'Bardia'): 0.0001059, ('B', 'Brad'): 0.0040425}


,ID,Title,1st author,repo,year,abstract,round-A_Pouria_output,round-A_Pouria_score,round-A_Pouria_reasoning,round-A_Bardia_output,round-A_Bardia_score,round-A_Bardia_reasoning,round-B_Brad_output,round-B_Brad_score,round-B_Brad_reasoning
0,72,Patient-centered data science: an integrative ...,"Amoei, M.",arXiv,2024,"This study proposes a novel, integrative frame...",{'reasoning': 'The abstract explicitly mention...,1,The abstract explicitly mentions the use of la...,{'reasoning': 'The article discusses the use o...,2,The article discusses the use of large languag...,"{'reasoning': 'Upon examining the abstract, it...",2,"Upon examining the abstract, it does mention t..."
1,583,Surrogate assisted methods for the parameteris...,"Perumal, R.",arXiv,2020,Parameter calibration is a major challenge in ...,{'reasoning': 'The article does not discuss la...,2,The article does not discuss large language mo...,{'reasoning': 'The article discusses surrogate...,2,The article discusses surrogate assisted metho...,None,None,None
2,614,In silico trial to test COVID-19 candidate vac...,"Russo, G.",bioRxiv,2020,SARS-CoV-2 is a severe respiratory infection t...,{'reasoning': 'The article focuses on an in si...,2,The article focuses on an in silico trial plat...,{'reasoning': 'The article discusses an in sil...,2,The article discusses an in silico platform fo...,None,None,None
3,213,Natural Language Programming in Medicine: Admi...,"Vaid, A.",arXiv,2024,Background Generative Large Language Models (L...,{'reasoning': 'The article title and abstract ...,1,The article title and abstract clearly indicat...,{'reasoning': 'The article discusses the appli...,2,The article discusses the application of large...,{'reasoning': 'The core distinction in the rev...,1,The core distinction in the reviews by Pouria ...
4,443,First passage time analysis of spatial mutatio...,"Haughey, M.J.",bioRxiv,2022,The footprint left by early cancer dynamics on...,{'reasoning': 'This article focuses on colorec...,2,This article focuses on colorectal cancer and ...,{'reasoning': 'The article discusses spatial m...,2,The article discusses spatial mutation pattern...,None,None,None
5,413,Innovations in Integrating Machine Learning an...,"Sivakumar, N.",arXiv,2022,"Abbreviations, acronyms: ABM, agent-based mode...",{'reasoning': 'The abstract does not mention l...,2,The abstract does not mention large language m...,{'reasoning': 'The title and abstract do not m...,2,The title and abstract do not mention large la...,None,None,None
6,358,TEAMWORK UNDER EXTREME UNCERTAINTY: AI FOR POK...,"Sarantinos, N.R.",arXiv,2022,The highest grossing media franchise of all ti...,{'reasoning': 'The article does not discuss la...,2,The article does not discuss large language mo...,{'reasoning': 'This article focuses on AI agen...,2,This article focuses on AI agents for video ga...,None,None,None
7,370,Generating synthetic data with a mechanism-bas...,"Cockrell, C.",bioRxiv,2022,Machine learning (ML) and Artificial Intellige...,{'reasoning': 'The abstract does not mention l...,2,The abstract does not mention large language m...,{'reasoning': 'The article discusses generatin...,2,The article discusses generating synthetic dat...,None,None,None
8,708,Inferring the ground truth through crowdsourcing,"Char, J.P.",arXiv,2018,—Universally valid ground truth is almost impo...,{'reasoning': 'The abstract does not mention l...,2,The abstract does not mention large language m...,{'reasoning': 'The article discusses crowdsour...,2,The article discusses crowdsourcing for inferr...,None,None,None
9,742,Verification &Validation of Agent Based Simula...,"Niazi, M.A.",arXiv,2017,Agent Based Models are very popular in a numbe...,{'reasoning': 'The article does not discuss la...,2,The article does not discuss large language mo...,{'reasoning': 'The article discusses agent-bas...,2,The article discusses agent-based models and t...,None,None,None


In [11]:
for i, row in updated_data.iterrows():
    print(
        f"""
        Title: {row.Title}
        Abstract: {row.abstract}
        Pouria's score: {row["round-A_Pouria_score"]}
        Pouria's reasoning: {row["round-A_Pouria_reasoning"]}
        Bardia's score: {row["round-A_Bardia_score"]}
        Bardia's reasoning: {row["round-A_Bardia_reasoning"]}
        Brad's score: {None if "round-B_Brad_score" not in row else row["round-B_Brad_score"]}
        Brad's reasoning: {None if "round-B_Brad_reasoning" not in row else row["round-B_Brad_reasoning"]}
        """
    )


        Title: Patient-centered data science: an integrative framework for evaluating and predicting clinical outcomes in the digital health era
        Abstract: This study proposes a novel, integrative framework for patient-centered data science in the digital health era. We developed a multidimensional model that combines traditional clinical data with patient-reported outcomes, social determinants of health, and multi-omic data to create comprehensive digital patient representations. Our framework employs a multi-agent artificial intelligence approach, utilizing various machine learning techniques including large language models, to analyze complex, longitudinal datasets. The model aims to optimize multiple patient outcomes simultaneously while addressing biases and ensuring generalizability. We demonstrate how this framework can be implemented to create a learning healthcare system that continuously refines strategies for optimal patient care. This approach has the potential to s